In [55]:
import numpy as np
import pandas as pd
from openbb_terminal.sdk import openbb
from pathlib import Path
import requests
import quandl
quandl.save_key("M2gMxzy1sXs7exUXrq8o")
print(quandl.ApiConfig.api_key)
from io import BytesIO
import seaborn as sns

M2gMxzy1sXs7exUXrq8o


In [51]:
sns.set_style('whitegrid')
idx = pd.IndexSlice

In [52]:
DATA_STORE=Path('assets.h5')

In [53]:
START = '2018-07-31'
END = '2023-07-31'

In [19]:
sp500_overview = openbb.stocks.screener.screener_data(preset_loaded='sp500_filter.ini', data_type = 'overview')
sp500_ownership = openbb.stocks.screener.screener_data(preset_loaded='sp500_filter.ini', data_type = 'ownership')
sp500_performance = openbb.stocks.screener.screener_data(preset_loaded='sp500_filter.ini', data_type = 'performance')
sp500_technical = openbb.stocks.screener.screener_data(preset_loaded='sp500_filter.ini', data_type = 'technical')
sp500_valuation = openbb.stocks.screener.screener_data(preset_loaded='sp500_filter.ini', data_type = 'valuation')

sp500_overview = sp500_overview.convert_dtypes()
sp500_ownership = sp500_ownership.convert_dtypes()
sp500_performance = sp500_performance.convert_dtypes()
sp500_technical = sp500_technical.convert_dtypes()
sp500_valuation = sp500_valuation.convert_dtypes()

sp500_overview.drop(columns = ['P/E'], inplace = True)
sp500_overview.set_index(keys = ['Ticker', 'Price', 'Change', 'Volume'], inplace = True)
sp500_performance.drop(columns = ['Avg Volume', 'Price', 'Change', 'Volume'], inplace = True)
sp500_performance.set_index(keys = ['Ticker'], inplace = True)
sp500_ownership.drop(columns = ['Price', 'Change', 'Volume', '\n\nMarket Cap'], inplace = True)
sp500_ownership.set_index(keys = ['Ticker'], inplace = True)
sp500_technical.drop(columns = ['Price', 'Change', 'Volume'], inplace = True)
sp500_technical.set_index(keys = ['Ticker'], inplace = True)
sp500_valuation.drop(columns = ['Price', 'Change', 'Volume', '\n\nMarket Cap'], inplace = True)
sp500_valuation.set_index(keys = ['Ticker'], inplace = True)

sp500_df = sp500_overview.join(sp500_valuation)
sp500_df = sp500_df.join(sp500_ownership)
sp500_df = sp500_df.join(sp500_performance)
sp500_df = sp500_df.join(sp500_technical)

sp500_df.reset_index(inplace = True)

sp500_df

,Ticker,Price,Change,Volume,Company,Sector,Industry,Country,\n\nMarket Cap,P/E,...,Beta,ATR,SMA20,SMA50,SMA200,52W High,52W Low,RSI,from Open,Gap
0,AAPL,182.9,-0.0358,72658572,Apple Inc.,Technology,Consumer Electronics,USA,2835.50B,30.74,...,1.27,3.46,0.0151,-0.0207,0.118,-0.0773,0.4731,48.88,-0.0288,-0.0072
1,MSFT,332.98,-0.0017,12774148,Microsoft Corporation,Technology,Software - Infrastructure,USA,2467.19B,34.37,...,0.9,6.18,0.0279,0.0006,0.1517,-0.0924,0.5597,56.53,-0.0015,-0.0005
2,GOOGL,134.47,-0.0096,16061402,Alphabet Inc.,Communication Services,Internet Content & Information,USA,1708.69B,29.44,...,1.06,2.98,0.0241,0.062,0.2373,-0.0257,0.6134,60.13,-0.0114,0.0018
3,GOOG,135.39,-0.0097,13616709,Alphabet Inc.,Communication Services,Internet Content & Information,USA,1699.45B,27.51,...,1.06,2.92,0.0255,0.064,0.2391,-0.0232,0.6222,60.27,-0.0126,0.0029
4,AMZN,135.38,-0.0138,34339595,"Amazon.com, Inc.",Consumer Cyclical,Internet Retail,USA,1395.71B,107.6,...,1.24,3.08,-0.0052,0.0174,0.238,-0.0576,0.6623,51.43,-0.0056,-0.0084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,OGN,19.83,-0.0557,3164356,Organon & Co.,Healthcare,Drug Manufacturers - General,USA,5.03B,6.72,...,0.77,0.6,-0.1179,-0.082,-0.176,-0.3885,0.0509,30.52,-0.0494,-0.0067
489,ZION,34.82,-0.0363,2938614,"Zions Bancorporation, National Association",Financial,Banks - Regional,USA,5.03B,6.12,...,1.14,1.33,-0.0228,0.0239,-0.0889,-0.4172,0.9069,48.08,-0.0219,-0.0147
490,LNC,25.52,0.0071,2427544,Lincoln National Corporation,Financial,Insurance - Life,USA,4.24B,<NA>,...,1.81,0.79,-0.0168,-0.0341,-0.0645,-0.5325,0.3795,45.07,0.0139,-0.0067
491,DXC,21.02,-0.0068,1883474,DXC Technology Company,Technology,Information Technology Services,USA,4.24B,<NA>,...,1.97,0.57,0.0247,-0.1301,-0.1862,-0.3054,0.1298,41.09,0.0026,-0.0094


In [ ]:
sp500_df.to_csv('test', encoding='utf-8')